In [10]:
import openpyxl
import re

In [11]:
data_wk = openpyxl.load_workbook('Thematic Taxonomy GPN.xlsx')
skos_wk = openpyxl.Workbook()

Setting the General parameters for the generated excel file

In [12]:
conceptSchemeURI = "http://data.undp.org/taxonomies/capacity/version01"
prefix = "capacity"
prefixURI = "http://data.undp.org/taxonomies/capacity/version01/"
title = "Capacity"
description = "UNDP Capacity Taxonomy"

comment = "This is a general comment that will be ignored. The converter will detect automatically the line containing the column titles below, so you can have a header as long as you want."

skos_wk_filename = "UNDP_Capacity_Tag_Taxonomy_Generated.xlsx"

debug = False

Generating the template structure with general populated fields

In [13]:
skos_sheet = skos_wk.active
skos_sheet["A1"] = "ConceptScheme URI"
skos_sheet["A2"] = "PREFIX"
skos_sheet["A3"] = "dct:title"
skos_sheet["A4"] = "dct:description"
skos_sheet["A6"] = comment

skos_sheet["B1"] = conceptSchemeURI
skos_sheet["B2"] = prefix
skos_sheet["B3"] = title
skos_sheet["B4"] = description

skos_sheet["C2"] = prefixURI

skos_sheet["A8"] = "URI"
skos_sheet["B8"] = "skos:prefLabel@en"
skos_sheet["C8"] = "skos:definition"
skos_sheet["D8"] = "skos:narrower"
skos_sheet["E8"] = "skos:notation"


Starting the processing of the information to translate the flat hierarchy to the template format

In [14]:
def uri_generator(value: str) -> str:
    res = value.strip()
    lowered = res.lower()
    replaced1 = re.sub("[^a-zA-Z0-9\s/]+","", lowered)
    replaced2 = re.sub("\s+", "_", replaced1)
    replaced3 = re.sub(",+", "", replaced2)
    res = replaced3
    return res

In [15]:
data_sheet = data_wk.active
print(f"Working with the sheet : {data_sheet}")
dimensions = data_sheet.dimensions
print(f"Range of data cells : {dimensions}")
firstCoordinate, secondCoordinate = dimensions.split(":")
numberOfRows = int(secondCoordinate[1:(len(secondCoordinate))])

columnsList = list(data_sheet.column_dimensions.keys())

cellValues = dict()
uriCollection = list()

for row in range(2, numberOfRows):
    rowPath = dict()
    i = 0
    for column in columnsList:
        
        data = data_sheet[f"{column}{row}"].value.strip()
        rowPath[column] = re.sub("/", "", data)
        value = "/".join(rowPath.values())
        uri = uri_generator(value)
        if uri not in uriCollection:
            cellValues[f"{uri}-{column}"] = { "uri": f"{prefix}:{uri}", "conceptPrefLabel": data, "narrower": dict() }
            uriCollection.append(f"{uri}")
    
        if column != columnsList[len(columnsList) - 1]:
            nextColumn = columnsList[i + 1]
            dataChild = data_sheet[f"{nextColumn}{row}"].value.strip()
            if dataChild is not None:
                rowPath[nextColumn] = re.sub("/", "", dataChild)
                value = "/".join(rowPath.values())
                uriChild = f"{prefix}:{uri_generator(value)}"
                if uriChild not in uriCollection:
                    cellValues[f"{uri}-{column}"]["narrower"][uriChild] = dataChild
        i += 1

print(f"Number of loaded Concept nodes : {len(cellValues)}")

i = 9
for k, v in cellValues.items(): 
    skos_sheet[f"A{i}"] = v["uri"]
    skos_sheet[f"B{i}"] = v["conceptPrefLabel"]
    skos_sheet[f"C{i}"] = v["conceptPrefLabel"]
    skos_sheet[f"D{i}"] = ", ".join(list(v["narrower"].keys()))
    skos_sheet[f"E{i}"] = v["conceptPrefLabel"]
    i += 1

Working with the sheet : <Worksheet "UNDP Capacity Taxonomy">
Range of data cells : A1:C325
Number of loaded Concept nodes : 345


In [16]:
skos_wk.save(skos_wk_filename)
skos_wk.close()
data_wk.close()